In [15]:
import pandas as pd

from src.constant import MAIN_DIR
from src.database import DB
from tqdm.auto import tqdm

PHASE2_DIR = MAIN_DIR / "archive" / "phase2"
N_TRAIN_LIST = ["25", "100"]
POLICY_LIST = ["baseline", "ea", "eb", "ec", "ia", "ib"]

In [16]:
records = []
total_iterations = len(N_TRAIN_LIST) * len(POLICY_LIST)
progress_bar = tqdm(total=total_iterations, desc="Processing")

for n_train in N_TRAIN_LIST:
    for policy in POLICY_LIST:
        # Update description to show current n_train and policy
        progress_bar.set_description(f"n_train={n_train}, policy={policy}")

        db_path_list = list((PHASE2_DIR / n_train / policy).glob("run-policy-*.db"))

        for db_path in db_path_list:
            db = DB(db_path)
            results = db.get_results()
            results_test = results.loc[results["prefix"].str.startswith("test")]
            if results_test.empty:
                print(f"No results {db_path}")
                continue
            cost = (
                results.loc[results["prefix"].str.startswith("test")]
                .groupby(["instance_id", "prefix"])["cost"]
                .min()  # min for every problem (of 2 solvers)
                .reset_index()
                .groupby("instance_id")["cost"]
                .median()  # median over 5 runs
                .mean()  # total mean score
            )
            cpu_time = (
                results.loc[
                    results["prefix"].str.startswith("config")
                    & results["cached"].eq(0)
                    & results["surrogate"].eq(0),
                    "time",
                ].sum()
                / 3600
            )
            surrogate_pct = results.loc[results["prefix"].str.startswith("config")].groupby(["solver_id", "instance_id"])["surrogate"].max().mean()
            real_pct = results.loc[results["prefix"].str.startswith("config")].groupby(["solver_id", "instance_id"])["surrogate"].min().eq(0).mean()
            records.append(
                {
                    "db_path": "/".join(db_path.parts[-3:]),
                    "n_train": n_train,
                    "policy": policy,
                    "cost": cost,
                    "cpu_time": cpu_time,
                    "surrogate_pct": surrogate_pct,
                    "real_pct": real_pct,
                }
            )
        progress_bar.update(1)

progress_bar.close()

df = pd.DataFrame(records)
df

Processing:   0%|          | 0/12 [00:00<?, ?it/s]

,db_path,n_train,policy,cost,cpu_time,surrogate_pct,real_pct
0,25/baseline/run-policy-baseline-25-1012821.db,25,baseline,0.20404,1.085407,0.000000,1.000000
1,25/baseline/run-policy-baseline-25-1012829.db,25,baseline,0.18832,1.231176,0.000000,1.000000
2,25/baseline/run-policy-baseline-25-1012830.db,25,baseline,0.73652,1.443054,0.000000,1.000000
3,25/baseline/run-policy-baseline-25-1012832.db,25,baseline,0.30916,0.832994,0.000000,1.000000
4,25/baseline/run-policy-baseline-25-1013020.db,25,baseline,0.20116,1.118927,0.000000,1.000000
...,...,...,...,...,...,...,...
100,100/ib/run-policy-ib-100-1013195.db,100,ib,0.29312,1.845759,0.918033,0.475410
101,100/ib/run-policy-ib-100-1013261.db,100,ib,0.54144,2.299865,0.907407,0.685185
102,100/ib/run-policy-ib-100-1013267.db,100,ib,0.15760,1.967770,0.939024,0.402439
103,100/ib/run-policy-ib-100-1013295.db,100,ib,0.38212,1.128463,0.924242,0.227273


In [17]:
df.pivot_table(index="policy", columns="n_train", values="cpu_time", aggfunc="count")

n_train,100,25
policy,,
baseline,9,9
ea,9,9
eb,9,9
ec,6,9
ia,9,9
ib,9,9


In [18]:
df.pivot_table(index="policy", columns="n_train", values="cpu_time", aggfunc="mean")

n_train,100,25
policy,,
baseline,4.403321,1.070346
ea,3.184109,0.816572
eb,2.066793,0.587298
ec,12.555337,3.044383
ia,3.069852,0.742710
ib,1.721727,0.410029


In [20]:
df.pivot_table(index="policy", columns="n_train", values="cost", aggfunc="mean")

n_train,100,25
policy,,
baseline,0.269547,0.324160
ea,0.312129,0.349618
eb,0.366613,0.361996
ec,0.262893,0.270084
ia,0.263249,0.243920
ib,0.368120,0.354658


In [21]:
def agg(x):
    df_agg = x.groupby("policy").agg(
        cost=("cost", "mean"),
        cpu_time=("cpu_time", "mean"),
        surrogate_pct=("surrogate_pct", "mean"),
        real_pct=("real_pct", "mean"),
    )
    df_agg["cost_ratio_to_baseline"] = df_agg["cost"] / df_agg.at["baseline", "cost"]
    df_agg["cpu_time_ratio_to_baseline"] = df_agg["cpu_time"] / df_agg.at["baseline", "cpu_time"]
    df_agg = df_agg.round(4)
    df_agg = df_agg.loc[:, ["cost", "cost_ratio_to_baseline", "cpu_time", "cpu_time_ratio_to_baseline", "surrogate_pct", "real_pct"]]
    return df_agg

df25 = df.loc[df["n_train"] == "25"].copy()
df100 = df.loc[df["n_train"] == "100"].copy()

df25_agg = agg(df25)
df100_agg = agg(df100)

In [14]:
df25_agg.to_excel("tmp.xlsx")

In [13]:
df100_agg

,cost,cost_ratio_to_baseline,cpu_time,cpu_time_ratio_to_baseline,surrogate_pct,real_pct
policy,,,,,,
baseline,0.2695,1.0000,4.4033,1.0000,0.0000,1.0000
ea,0.3121,1.1580,3.1841,0.7231,0.4659,0.6622
eb,0.3666,1.3601,2.0668,0.4694,0.9313,0.6615
ec,0.2629,0.9753,12.5553,2.8513,0.9243,1.0000
ia,0.2632,0.9766,3.0699,0.6972,0.4463,0.7076
ib,0.3681,1.3657,1.7217,0.3910,0.9237,0.3928


In [22]:
db = DB(PHASE2_DIR / "25/ec/run-policy-ec-25-1013473.db")
results = db.get_results()

In [24]:
results.loc[lambda x: x["surrogate"].eq(1)]

,id,prefix,solver_id,instance_id,cost,time,cut_off_cost,cut_off_time,cached,surrogate,error
125,config;solver=1;attempt=1;aac_iter=6;surrogate...,config;solver=1;attempt=1;aac_iter=6;surrogate,1618467634991678120,1199808321398786303,2.843311,0.0,7.4,0.74,0,1,0
126,config;solver=1;attempt=1;aac_iter=6;surrogate...,config;solver=1;attempt=1;aac_iter=6;surrogate,1618467634991678120,926855222569918425,3.07088,0.0,77.7,7.77,0,1,0
127,config;solver=1;attempt=1;aac_iter=6;surrogate...,config;solver=1;attempt=1;aac_iter=6;surrogate,1618467634991678120,2036633111162358606,2.842577,0.0,13.3,1.33,0,1,0
128,config;solver=1;attempt=1;aac_iter=6;surrogate...,config;solver=1;attempt=1;aac_iter=6;surrogate,1618467634991678120,1493964260327799128,3.034269,0.0,25.4,2.54,0,1,0
129,config;solver=1;attempt=1;aac_iter=6;surrogate...,config;solver=1;attempt=1;aac_iter=6;surrogate,1618467634991678120,5624699322667089,2.632905,0.0,13.9,1.39,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
5766,config;solver=2;attempt=1;aac_iter=25;surrogat...,config;solver=2;attempt=1;aac_iter=25;surrogate,228607822512541209,622705132865616942,b'\x92\x8d*A',0.0,15.1,1.51,0,1,0
5768,config;solver=2;attempt=1;aac_iter=25;surrogat...,config;solver=2;attempt=1;aac_iter=25;surrogate,228607822512541209,1027898753954028458,b'o\x97\xcc?',0.0,1.5,0.15,0,1,0
5770,config;solver=2;attempt=1;aac_iter=25;surrogat...,config;solver=2;attempt=1;aac_iter=25;surrogate,228607822512541209,1084422477911079854,b's\x8cq@',0.0,3.5,0.35,0,1,0
5772,config;solver=2;attempt=1;aac_iter=25;surrogat...,config;solver=2;attempt=1;aac_iter=25;surrogate,228607822512541209,1337376689788754595,b'\xd5\x1f\x91@',0.0,3.6,0.36,0,1,0


In [26]:
df.loc[(df["policy"] == "ia") & (df["n_train"] == "25")].sort_values(by="cost")

,db_path,n_train,policy,cost,cpu_time,surrogate_pct,real_pct
43,25/ia/run-policy-ia-25-1013301.db,25,ia,0.11736,0.746655,0.435484,0.709677
36,25/ia/run-policy-ia-25-1012849.db,25,ia,0.16352,0.873884,0.451220,0.658537
39,25/ia/run-policy-ia-25-1012853.db,25,ia,0.16744,0.740041,0.437500,0.687500
40,25/ia/run-policy-ia-25-1013024.db,25,ia,0.20272,0.754318,0.442623,0.721311
42,25/ia/run-policy-ia-25-1013054.db,25,ia,0.20496,0.855532,0.466667,0.666667
38,25/ia/run-policy-ia-25-1012852.db,25,ia,0.23492,0.611252,0.425926,0.740741
37,25/ia/run-policy-ia-25-1012851.db,25,ia,0.33148,0.714736,0.456140,0.736842
41,25/ia/run-policy-ia-25-1013030.db,25,ia,0.38092,0.586128,0.437500,0.750000
44,25/ia/run-policy-ia-25-1013307.db,25,ia,0.39196,0.801841,0.435484,0.709677
